In [1]:
import sqlite3
from tabulate import tabulate
conn = sqlite3.connect("../enquete.db")
c = conn.cursor()

In [2]:
suspects = list(range(5001))

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

In [3]:
query_vacances = """
SELECT id
FROM employés
INNER JOIN congés_2023 ON employés.id = congés_2023.id_employé
WHERE "2023-04-01" BETWEEN  date_départ_accordée AND date_retour_accordée
"""
disculpés = c.execute(query_vacances)
disculpés = [elem[0]for elem in disculpés.fetchall()]
print("Nombre de personnes absentes le jour du meurtre :",len(disculpés))

Nombre de personnes absentes le jour du meurtre : 65


In [4]:
suspects = [suspect for suspect in suspects if suspect not in disculpés]
print("Nombre de personnes suspectes :",len(suspects))

Nombre de personnes suspectes : 4936


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

In [5]:

!rm mails_boss.md
from Mail import Mail

query_mails = """
SELECT * FROM mails
WHERE id_expéditeur NOT IN (
    SELECT id
    FROM employés
    INNER JOIN congés_2023 ON employés.id = congés_2023.id_employé
    WHERE "2023-04-01" BETWEEN  date_départ_accordée AND date_retour_accordée
)
"""

mails = c.execute(query_mails)

mails = mails.fetchall()

for line in mails:
    mail = Mail(line)
    regex_boss = r"PDG|[Bb]oss|[Pp]atron|Jean Dubois"
    mail.find_regex_in_mails(regex_boss, "mails_boss.md")

!open  mails_boss.md

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

In [3]:
query_employés_louches = """
SELECT * FROM employés
WHERE id IN (2309,728,3789)
"""
employés_louches = c.execute(query_employés_louches)
employés_louches = employés_louches.fetchall()
headers = ["id","nom","prénom","fonction","salaire","date d'embauche","id_projet"]
print(tabulate(employés_louches,headers,tablefmt="fancy_grid"))


╒══════╤══════════╤══════════╤═══════════════╤═══════════╤═══════════════════╤═════════════╕
│   id │ nom      │ prénom   │ fonction      │   salaire │ date d'embauche   │   id_projet │
╞══════╪══════════╪══════════╪═══════════════╪═══════════╪═══════════════════╪═════════════╡
│  340 │ Coupable │ John     │ Ingénieur TAL │      2000 │ 2012-04-01        │           4 │
╘══════╧══════════╧══════════╧═══════════════╧═══════════╧═══════════════════╧═════════════╛


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

<br></br>
<br></br>

In [7]:
projet4 = c.execute("SELECT * FROM projets WHERE id_projet = 4")
projet4 = projet4.fetchall()
headers = ["id projet","nom projet","description projet","date début projet","id chef de projet"]
print(tabulate(projet4,headers,tablefmt="fancy_grid"))

╒═════════════╤══════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════╤═════════════════════╕
│   id projet │ nom projet   │ description projet                                                                                          │ date début projet   │   id chef de projet │
╞═════════════╪══════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════════════════╪═════════════════════╪═════════════════════╡
│           4 │ TitanAI      │ Projet de LLM visant à élaborer une IA travailleuse qui maîtrise le développement d'applications et de LLM. │ 2009-04-23          │                3809 │
╘═════════════╧══════════════╧═════════════════════════════════════════════════════════════════════════════════════════════════════════════╧═════════════════════╧═════════════════════╛


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

In [8]:
query_nombre_employes_projet = """
SELECT COUNT(*)
FROM employés
WHERE id_projet = 4
AND employés.id NOT in (SELECT id
                        FROM employés
                        INNER JOIN congés_2023
                        ON employés.id = congés_2023.id_employé
                        WHERE "2023-04-01" BETWEEN date_départ_accordée AND date_retour_accordée);
"""

nombre_employés = c.execute(query_nombre_employes_projet)
nombre_employés = nombre_employés.fetchone()
print("Nombre d'employés dans le projet TitanAI : ",nombre_employés[0])

Nombre d'employés dans le projet TitanAI :  9


In [11]:
query_infos_employes_projet = """
SELECT *
FROM employés
WHERE id_projet = 4
AND employés.id NOT in (SELECT id
                        FROM employés
                        INNER JOIN congés_2023
                        ON employés.id = congés_2023.id_employé
                        WHERE "2023-04-01" BETWEEN date_départ_accordée AND date_retour_accordée);
"""

membres = c.execute(query_infos_employes_projet)
membres = membres.fetchall()
headers = ["id","nom","prénom","fonction","salaire","date d'embauche","id_projet"]
print(tabulate(membres,headers,tablefmt="fancy_grid"))


╒══════╤══════════╤═══════════╤════════════════════╤═══════════╤═══════════════════╤═════════════╕
│   id │ nom      │ prénom    │ fonction           │   salaire │ date d'embauche   │   id_projet │
╞══════╪══════════╪═══════════╪════════════════════╪═══════════╪═══════════════════╪═════════════╡
│   18 │ Lemaire  │ Amandine  │ Experte en ML      │     10000 │ 2004-10-23        │           4 │
├──────┼──────────┼───────────┼────────────────────┼───────────┼───────────────────┼─────────────┤
│  728 │ Dengrot  │ Drécic    │ Speech Analyst     │     70000 │ 2022-09-18        │           4 │
├──────┼──────────┼───────────┼────────────────────┼───────────┼───────────────────┼─────────────┤
│ 1784 │ Fleith   │ Florian   │ Employé polyvalent │     10000 │ 2001-03-12        │           4 │
├──────┼──────────┼───────────┼────────────────────┼───────────┼───────────────────┼─────────────┤
│ 2000 │ Philippe │ Ashley    │ Développeur        │      7000 │ 1998-11-24        │           4 │
├──────┼──

In [8]:
query_salaires_employes_projet = """
SELECT fonction, COUNT(fonction) AS nb, AVG(salaire) AS salaire_moyen
FROM employés
WHERE id_projet = 4
AND employés.id NOT in (SELECT id
                        FROM employés
                        INNER JOIN congés_2023
                        ON employés.id = congés_2023.id_employé
                        WHERE "2023-04-01" BETWEEN date_départ_accordée AND date_retour_accordée)
GROUP BY fonction;
"""
# RAJOUTER SALAIRE MOYEN DANS L'ENTREPRISE POUR COMPARER
salaires = c.execute(query_salaires_employes_projet)
salaires = salaires.fetchall()
headers = ["Poste","Nombre d'employés","Salaire Moyen"]
print(tabulate(salaires,headers,tablefmt="fancy_grid"))

╒════════════════════╤═════════════════════╤═════════════════╕
│ Poste              │   Nombre d'employés │   Salaire Moyen │
╞════════════════════╪═════════════════════╪═════════════════╡
│ Chef de projet     │                   1 │          150000 │
├────────────────────┼─────────────────────┼─────────────────┤
│ Développeur        │                   3 │            7500 │
├────────────────────┼─────────────────────┼─────────────────┤
│ Employé polyvalent │                   1 │           10000 │
├────────────────────┼─────────────────────┼─────────────────┤
│ Experte en ML      │                   1 │           10000 │
├────────────────────┼─────────────────────┼─────────────────┤
│ Ingénieur TAL      │                   2 │           11400 │
├────────────────────┼─────────────────────┼─────────────────┤
│ Speech Analyst     │                   1 │           70000 │
╘════════════════════╧═════════════════════╧═════════════════╛


<br></br><br></br><br></br><br></br><br></br><br></br><br></br><br></br><br></br>


In [14]:
!rm mails_project.md
for line in mails:
    mail = Mail(line)
    regex_project = "((projet)?[tT]itan[aA][iI])"
    # regex_project = "Titan"
    mail.find_regex_in_mails(regex_project, "mails_project.md")

!open mails_project.md

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

In [15]:
#À adapter
query_caméras_badges = """
SELECT nom, prénom, lieu_badge,date_heure_badge,type_badge,lieu,heure,action
FROM employés
INNER JOIN reconnaissance_caméras_2023_04_01 on employés.id = reconnaissance_caméras_2023_04_01.id_employé
INNER JOIN badges_2023 ON employés.id = badges_2023.id_employé
WHERE date_heure_badge LIKE "2023-04-01|14:5%"
AND heure LIKE "14:5%"
AND employés.id IN (728,2309,3789)"""

caméras_badges = c.execute(query_caméras_badges)
caméras_badges = caméras_badges.fetchall()
headers = ["nom","prénom","lieu badge","heure badge","type badge","lieu caméra","heure caméra","action caméra"]
print(tabulate(caméras_badges,headers,tablefmt="fancy_grid"))


# Idees de requetes en plus si on a le temps : ceux qui ont demande des conges mais qui n'ont jamais ete accordes
# faire un truc autour de badges : qui a ete dans le bureau du PDG ? Coupable y va h24

╒══════════╤══════════╤═════════════════╤═════════════════════╤══════════════╤═════════════════╤════════════════╤═══════════════════╕
│ nom      │ prénom   │ lieu badge      │ heure badge         │ type badge   │ lieu caméra     │ heure caméra   │ action caméra     │
╞══════════╪══════════╪═════════════════╪═════════════════════╪══════════════╪═════════════════╪════════════════╪═══════════════════╡
│ Dengrot  │ Drécic   │ Cour intérieure │ 2023-04-01|14:58:12 │ Sortie       │ Cour intérieure │ 14:58:54       │ En train de fumer │
├──────────┼──────────┼─────────────────┼─────────────────────┼──────────────┼─────────────────┼────────────────┼───────────────────┤
│ Dupont   │ Pierre   │ Cour intérieure │ 2023-04-01|14:58:28 │ Sortie       │ Cour intérieure │ 14:58:20       │ En train de fumer │
├──────────┼──────────┼─────────────────┼─────────────────────┼──────────────┼─────────────────┼────────────────┼───────────────────┤
│ Mariotte │ Laurent  │ Cour intérieure │ 2023-04-01|14:58:01 

In [11]:
query_caméras_badges = """
SELECT nom, prénom, lieu_badge,date_heure_badge,type_badge,lieu,heure,action
FROM employés
INNER JOIN reconnaissance_caméras_2023_04_01 on employés.id = reconnaissance_caméras_2023_04_01.id_employé
INNER JOIN badges_2023 ON employés.id = badges_2023.id_employé
WHERE date_heure_badge LIKE "2023-04-01|15:%"
AND heure LIKE "15:%"
AND employés.id IN (728,2309,3789)"""

caméras_badges = c.execute(query_caméras_badges)
caméras_badges = caméras_badges.fetchall()
headers = ["nom","prénom","lieu badge","heure badge","type badge","lieu caméra","heure caméra","action caméra"]
print(tabulate(caméras_badges,headers,tablefmt="fancy_grid"))

╒══════════╤══════════╤══════════════╤═════════════════════╤══════════════╤═══════════════╤════════════════╤══════════════════╕
│ nom      │ prénom   │ lieu badge   │ heure badge         │ type badge   │ lieu caméra   │ heure caméra   │ action caméra    │
╞══════════╪══════════╪══════════════╪═════════════════════╪══════════════╪═══════════════╪════════════════╪══════════════════╡
│ Dengrot  │ Drécic   │ Open Space   │ 2023-04-01|15:09:28 │ Entrée       │ Open Space    │ 15:10:34       │ Sur l'ordinateur │
├──────────┼──────────┼──────────────┼─────────────────────┼──────────────┼───────────────┼────────────────┼──────────────────┤
│ Dupont   │ Pierre   │ Open Space   │ 2023-04-01|15:09:35 │ Entrée       │ Open Space    │ 15:11:07       │ Sur l'ordinateur │
├──────────┼──────────┼──────────────┼─────────────────────┼──────────────┼───────────────┼────────────────┼──────────────────┤
│ Mariotte │ Laurent  │ Open Space   │ 2023-04-01|15:09:42 │ Entrée       │ Open Space    │ 15:11:02    

In [3]:
c.execute("SELECT date_départ_accordée,date_retour_accordée,id_employé FROM congés_2023 WHERE id_employé = 340")
res = c.fetchall()
headers = ["Date départ","Date retour","id"]
print(tabulate(res,headers,tablefmt="fancy_grid"))

╒═══════════════╤═══════════════╤══════╕
│ Date départ   │ Date retour   │   id │
╞═══════════════╪═══════════════╪══════╡
│ 2023-04-01    │ 2023-04-02    │  340 │
╘═══════════════╧═══════════════╧══════╛


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

In [4]:
c.execute("SELECT date_départ_accordée,date_retour_accordée,id_employé FROM congés_2023 WHERE id_employé = 340")
congé = c.fetchall()
headers = ["date départ","date retour","id"]
print(tabulate(congé,headers,tablefmt="fancy_grid"))

╒═══════════════╤═══════════════╤══════╕
│ date départ   │ date retour   │   id │
╞═══════════════╪═══════════════╪══════╡
│ 2023-04-01    │ 2023-04-02    │  340 │
╘═══════════════╧═══════════════╧══════╛
